# clean

> This module contains all the various cleaning options supported.

In [ ]:
# | default_exp clean

In [ ]:
# | export
import re
from faker import Faker
import ftfy

fake = Faker()

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
# From: https://github.com/bigscience-workshop/data-preparation/blob/main/preprocessing/training/01b_oscar_cleaning_and_filtering/filtering.py#L95
whitespace = {
    " ",
    " ",
    " ",
    " ",
    " ",
    "　",
    " ",
    " ",
    " ",
    " ",
    "￼",
    "",
}


def normalize_whitespace(
    text: str,  # The text to normalize
) -> str:  # The normalized text
    """
    Replace the various whitespace characters with the standard one.
    """
    text = "".join([char if char not in whitespace else " " for char in text])
    return text

In [ ]:
# test the normalize_whitespace function
assert normalize_whitespace("a b c d e　f g h i￼jk") == "a b c d e f g h i j k"

In [ ]:
# | export
unicode_punctuation = {
    "，": ",",
    "。": ".",
    "、": ",",
    "„": '"',
    "”": '"',
    "“": '"',
    "«": '"',
    "»": '"',
    "１": '"',
    "」": '"',
    "「": '"',
    "《": '"',
    "》": '"',
    "´": "'",
    "∶": ":",
    "：": ":",
    "？": "?",
    "！": "!",
    "（": "(",
    "）": ")",
    "；": ";",
    "–": "-",
    "—": " - ",
    "．": ". ",
    "～": "~",
    "’": "'",
    "…": "...",
    "━": "-",
    "〈": "<",
    "〉": ">",
    "【": "[",
    "】": "]",
    "％": "%",
    "►": "-",
}


def normalize_punctuation(
    text: str,  # The text to normalize
) -> str:  # The normalized text
    """
    Replace the various unicode punctuation characters with the standard ones.
    """
    text = "".join([unicode_punctuation.get(char, char) for char in text])
    return text

In [ ]:
# test the normalize_punctuation function
text = "，。、„”“«»１」「《》´∶：？！（）；–—．～’…━〈〉【】％►"

assert normalize_punctuation(text) == ',.,""""""""""\'::?!();- - . ~\'...-<>[]%-'

In [ ]:
# | export
def remove_empty_lines(
    text: str,  # The text to remove empty lines from
) -> str:  # The text with empty lines removed
    """
    Remove empty lines from the text.
    Solution from https://stackoverflow.com/a/3711884/5768407
    """
    lines = text.splitlines()
    filtered = filter(lambda x: not re.match(r"^\s*$", x), lines)
    return "\n".join(filtered)

In [ ]:
# test the remove_empty_lines function
starts_with_newline = "\nfoo\nbar"
multiple_newlines = "foo\n\nbar"
ends_with_newline = "foo\nbar\n"

assert remove_empty_lines(starts_with_newline) == "foo\nbar"
assert remove_empty_lines(multiple_newlines) == "foo\nbar"
assert remove_empty_lines(ends_with_newline) == "foo\nbar"

In [ ]:
# | export
def replace_urls(
    text: str,  # The text to replace URLs in
    dummy: str = "https://example.com/",  # The dummy text to replace URLs with
) -> str:  # The text with URLs replaced
    """Replace urls from text with a dummy."""
    return re.sub(r"http\S+", dummy, text)

In [ ]:
# test the replace_urls function
url_after_space = "foo http://bar.com"
url_before_space = "http://foo.com bar"
assert replace_urls(url_after_space) == "foo https://example.com/"
assert replace_urls(url_before_space) == "https://example.com/ bar"

In [ ]:
# | export
def replace_dates(
    text: str,  # The text to remove dates from
    dummy: str = fake.date(),  # The dummy text to replace dates with
) -> str:  # The text with dates replaced
    """Replace dates from text with a dummy."""
    return re.sub(r"\d{1,2}/\d{1,2}/\d{4}", dummy, text)

In [ ]:
# test the replace_dates function
date_after_space = "foo 1/1/2020"
date_before_space = "1/1/2020 bar"
assert replace_dates(date_after_space, "1/1/1970") == "foo 1/1/1970"
assert replace_dates(date_before_space, "1/1/1970") == "1/1/1970 bar"

## PII Removal

Currently, we support the following PII removal options:

  * `replace_email`
  * `replace_phone`
  * `replace_ip`
  * `replace_credit_card`
  * `replace_ssn`

However, for emails, phone numbers, credit cards, and SSNs, we recommend you to use the [scrubadub](https://scrubadub.readthedocs.io/en/stable/index.html) library.

In [ ]:
# | export
def replace_email(
    text: str,  # The text to replace email addresses in
    dummy: str = fake.email(),  # The dummy text to replace email addresses with
) -> str:  # The text with email addresses replaced
    """Replace email addresses from text with a dummy."""
    return re.sub(r"[\w\.-]+@[\w\.-]+", dummy, text)

In [ ]:
# test the replace_email function
email_after_space = "foo fake@email.com"
email_before_space = "fake@email.com bar"
email_with_forward_periods = "foo.bar@email.com"
email_with_backward_periods = "foo@bar.email.com"

assert replace_email(email_after_space, "example@email.com") == "foo example@email.com"
assert replace_email(email_before_space, "example@email.com") == "example@email.com bar"
assert (
    replace_email(email_with_forward_periods, "example@email.com")
    == "example@email.com"
)
assert (
    replace_email(email_with_backward_periods, "example@email.com")
    == "example@email.com"
)

In [ ]:
# | export
def replace_phone(
    text: str,  # The text to replace phone numbers in
    dummy: str = fake.phone_number(),  # The dummy text to replace phone numbers with
) -> str:  # The text with phone numbers replaced
    """Replace phone numbers from text with a dummy."""
    return re.sub(r"\(?\d{3}\)?-? *\d{3}-? *-?\d{4}", dummy, text)

In [ ]:
# test the replace_phone function
phone_after_space = "foo 111-222-3333"
phone_before_space = "111-222-3333 bar"
phone_with_parens = "(111) 222-3333"
phone_with_spaces = "111 222 3333"
phone_with_dashes = "111-222-3333"

assert replace_phone(phone_after_space, "123-456-7890") == "foo 123-456-7890"
assert replace_phone(phone_before_space, "123-456-7890") == "123-456-7890 bar"
assert replace_phone(phone_with_parens, "123-456-7890") == "123-456-7890"
assert replace_phone(phone_with_spaces, "123-456-7890") == "123-456-7890"
assert replace_phone(phone_with_dashes, "123-456-7890") == "123-456-7890"

In [ ]:
# | export
def replace_ip(
    text,  # The text to replace ip addresses in
    dummy1: str = fake.ipv4(),  # The dummy text to replace ipv4 addresses with
    dummy2: str = fake.ipv6(),  # The dummy text to replace ipv6 addresses with
) -> str:  # The text with ip addresses replaced
    """
    Replace ip addresses from text with a dummy.
    Solution from https://github.com/bigcode-project/bigcode-analysis/blob/main/data_analysis/pii/utils/emails_ip_addresses_detection.py#L48
    """
    ipv4_pattern = r"(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(?:\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}"
    text = re.sub(ipv4_pattern, dummy1, text)
    ipv6_pattern = r"(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])"
    text = re.sub(ipv6_pattern, dummy2, text)
    return text

In [ ]:
# test the replace_ip function
ip4_after_space = "foo 111.222.3.4"
ip4_before_space = "111.222.3.4 bar"
ip6_with_colons = "2001:0db8:0000:0000:0000:8a2e:0370:7334"

assert replace_ip(ip4_after_space, "127.0.0.1") == "foo 127.0.0.1"
assert replace_ip(ip4_before_space, "127.0.0.1") == "127.0.0.1 bar"
assert replace_ip(ip6_with_colons, "127.0.0.1", "0:0:0:0:0:0:0:1") == "0:0:0:0:0:0:0:1"

In [ ]:
# | export
def replace_credit_card(
    text: str,  # The text to replace credit card numbers in
    dummy: str = fake.credit_card_number(),  # The dummy text to replace credit card numbers with
) -> str:  # The text with credit card numbers replaced
    """Replace credit card numbers from text with a dummy."""
    return re.sub(r"\d{4}-\d{4}-\d{4}-\d{4}", dummy, text)

In [ ]:
# test the replace_credit_card function
credit_card_after_space = "foo 1111-2222-3333-4444"
credit_card_before_space = "1111-2222-3333-4444 bar"

assert (
    replace_credit_card(credit_card_after_space, "1234-5678-9012-3456")
    == "foo 1234-5678-9012-3456"
)
assert (
    replace_credit_card(credit_card_before_space, "1234-5678-9012-3456")
    == "1234-5678-9012-3456 bar"
)

In [ ]:
# | export
def replace_ssn(
    text: str,  # The text to replace social security numbers in
    dummy: str = fake.ssn(),  # The dummy text to replace social security numbers with
) -> str:  # The text with social security numbers replaced
    """Replace social security numbers from text with a dummy."""
    return re.sub(r"\d{3}-\d{2}-\d{4}", dummy, text)

In [ ]:
# test the replace_ssn function
ssn_after_space = "foo 111-22-3333"
ssn_before_space = "111-22-3333 bar"

assert replace_ssn(ssn_after_space, "123-45-6789") == "foo 123-45-6789"
assert replace_ssn(ssn_before_space, "123-45-6789") == "123-45-6789 bar"

In [ ]:
# | export
def fix_utf8_encoding(
    text: str,  # The text to fix
) -> str:  # The fixed text
    """Fix utf8 text using ftfy."""
    return ftfy.fix_text(text)

In [ ]:
# test the fix_utf8_encoding function
bad_text = "âœ” No problems"
assert fix_utf8_encoding(bad_text) == "✔ No problems"
bad_text = "dÃ©jÃ  vu"
assert fix_utf8_encoding(bad_text) == "déjà vu"
bad_text = "Ã©"
assert fix_utf8_encoding(bad_text) == "é"
bad_text = "P&EACUTE;REZ"
assert fix_utf8_encoding(bad_text) == "PÉREZ"

In [ ]:
# | export
def clean_code_license(
    code: str,  # The code to clean
    language: str = "python",  # The language of the code
    min_lines: int = 3,  # The minimum number of lines that need to be removed
):
    import code_ast
    from code_ast import ASTVisitor
    from code_ast.ast import LEAVE_WHITELIST

    class FirstNonCommentVisitor(ASTVisitor):
        def __init__(self):
            self.passed_global_node = False
            self.first_node = None

        def visit(self, node):
            if not self.passed_global_node:
                self.passed_global_node = True
                return
            if self.first_node is None:
                if node.child_count > 0 or node.type in LEAVE_WHITELIST:
                    self.first_node = node

    """Remove the license or other boilerplate comments from the code."""
    ast = code_ast.ast(code, lang=language)
    visitor = FirstNonCommentVisitor()
    ast.visit(visitor)
    start_line = visitor.first_node.start_point[0]
    if start_line < min_lines:
        return code
    else:
        return "\n".join(code.splitlines()[start_line:])

In [ ]:
# |eval: false
# Test the cleaning of code licenses or similar boilerplate comments from code
code_python = """# -*- coding: utf-8 -*-

# Copyright 2018 Spanish National Research Council (CSIC)
#
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not use this file except in compliance with the License. You may obtain
# a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.

\"\"\"
Given two dates and region, download N Sentinel Collections scenes from ESA
Sentinel dataHUB.
The downloaded Sentinel collection scenes are compatible with:
S2MSI1C: Top-of-atmosphere reflectances in cartographic geometry
or S2MSI2A: Bottom-of-atmosphere reflectance in cartographic geometry
Parameters
----------
inidate: datetime.strptime("YYYY-MM-dd", "%Y-%m-%d")
enddate: datetime.strptime("YYYY-MM-dd", "%Y-%m-%d")
region: name of one reservoir saved in the "coord_reservoirs.json" file
coordinates : dict. Coordinates of the region to search.
Example: {"W": -2.830, "S": 41.820, "E": -2.690, "N": 41.910}}
platform : str. Satellite to use from the Sentinel family
producttype : str. Dataset type.
cloud: int
path : path
Author: Daniel García Díaz
Email: garciad@ifca.unican.es
Institute of Physics of Cantabria (IFCA)
Advanced Computing and e-Science
Date: Sep 2018
\"\"\"
#imports apis
import requests
import os

# Subfunctions
from wq_sat.utils import config
"""

code_go = """// +build go1.9

// Copyright 2019 Microsoft Corporation
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//     http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

// This code was auto-generated by:
// github.com/Azure/azure-sdk-for-go/tools/profileBuilder

package policyinsights

import (
	"context"

	original "github.com/Azure/azure-sdk-for-go/services/policyinsights/mgmt/2019-10-01/policyinsights"
)
"""

code_c = """/*
 * copyright (c) 2008 - 2011 Espressif System
 *
 * Define user specified Event signals and Task priorities here
 *
 */

#ifndef _ETS_SYS_H
#define _ETS_SYS_H

#include "c_types.h"
#include "eagle_soc.h"

typedef uint32_t ETSSignal;
"""

code_cpp = """/*  Pokemon Automation Bot Base - Client Example

 * 

 *  From: https://github.com/PokemonAutomation/Arduino-Source

 * 

 */



#include "Common/CRC32.h"

#include "Common/Microcontroller/MessageProtocol.h"

#include "ClientSource/Libraries/Logging.h"

#include "ClientSource/Libraries/MessageConverter.h"

#include "BotBaseMessage.h"

#include "PABotBaseConnection.h"



#include <iostream>

using std::cout;
"""

code_java = """/*
 * Copyright (C) 2012-2021 DuyHai DOAN
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

package info.archinnov.achilles.internals.sample_classes.parser.entity;

import info.archinnov.achilles.annotations.Column;
"""

code_javascript = """/*
** Copyright (c) 2016-2019, Thomas Farr
**
** This Source Code Form is subject to the terms of the Mozilla Public
** License, v. 2.0. If a copy of the MPL was not distributed with this
** file, You can obtain one at https://mozilla.org/MPL/2.0/.
*/

// TODO: Implement testing of option handling, and filename arrays

const anitomy = require('../anitomy');
const async = require('async');
"""

cleaned_code_python = clean_code_license(code_python, language="python")
cleaned_code_go = clean_code_license(code_go, language="go")
cleaned_code_c = clean_code_license(code_c, language="c")
cleaned_code_cpp = clean_code_license(code_cpp, language="cpp")
cleaned_code_java = clean_code_license(code_java, language="java")
cleaned_code_javascript = clean_code_license(code_javascript, language="javascript")

assert (
    cleaned_code_python
    == """\"\"\"
Given two dates and region, download N Sentinel Collections scenes from ESA
Sentinel dataHUB.
The downloaded Sentinel collection scenes are compatible with:
S2MSI1C: Top-of-atmosphere reflectances in cartographic geometry
or S2MSI2A: Bottom-of-atmosphere reflectance in cartographic geometry
Parameters
----------
inidate: datetime.strptime("YYYY-MM-dd", "%Y-%m-%d")
enddate: datetime.strptime("YYYY-MM-dd", "%Y-%m-%d")
region: name of one reservoir saved in the "coord_reservoirs.json" file
coordinates : dict. Coordinates of the region to search.
Example: {"W": -2.830, "S": 41.820, "E": -2.690, "N": 41.910}}
platform : str. Satellite to use from the Sentinel family
producttype : str. Dataset type.
cloud: int
path : path
Author: Daniel García Díaz
Email: garciad@ifca.unican.es
Institute of Physics of Cantabria (IFCA)
Advanced Computing and e-Science
Date: Sep 2018
\"\"\"
#imports apis
import requests
import os

# Subfunctions
from wq_sat.utils import config"""
)
assert (
    cleaned_code_go
    == """package policyinsights

import (
	"context"

	original "github.com/Azure/azure-sdk-for-go/services/policyinsights/mgmt/2019-10-01/policyinsights"
)"""
)
assert (
    cleaned_code_c
    == """#ifndef _ETS_SYS_H
#define _ETS_SYS_H

#include "c_types.h"
#include "eagle_soc.h"

typedef uint32_t ETSSignal;"""
)
assert (
    cleaned_code_cpp
    == """#include "Common/CRC32.h"

#include "Common/Microcontroller/MessageProtocol.h"

#include "ClientSource/Libraries/Logging.h"

#include "ClientSource/Libraries/MessageConverter.h"

#include "BotBaseMessage.h"

#include "PABotBaseConnection.h"



#include <iostream>

using std::cout;"""
)
assert (
    cleaned_code_java
    == """package info.archinnov.achilles.internals.sample_classes.parser.entity;

import info.archinnov.achilles.annotations.Column;"""
)
assert (
    cleaned_code_javascript
    == """const anitomy = require('../anitomy');
const async = require('async');"""
)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()